In [123]:
import pandas as pd
import numpy as np
import requests

In [124]:
df = pd.read_csv('wildfires.csv', parse_dates=True)

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11915 entries, 0 to 11914
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   YEAR        11915 non-null  int64  
 1   AGENCY      11915 non-null  object 
 2   UNIT_ID     11912 non-null  object 
 3   FIRE_NAME   11887 non-null  object 
 4   INC_NUM     11495 non-null  object 
 5   ALARM_DATE  7992 non-null   object 
 6   CONT_DATE   3156 non-null   object 
 7   CAUSE       11904 non-null  object 
 8   REPORT_AC   3594 non-null   float64
 9   GIS_ACRES   11915 non-null  float64
 10  C_METHOD    3478 non-null   object 
 11  OBJECTIVE   11859 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 1.1+ MB


In [126]:
len(df.UNIT_ID.unique())
units = df.UNIT_ID.unique()

In [127]:
# curl "https://api.radar.io/v1/search/autocomplete?query=841+Broadway&near=40.70390%2C-73.98670" -H "Authorization: prj_live_pk_4f017205dfaad39af3182404866753da475d1e04"

In [128]:
'''
{'latitude': 34.05513,
 'longitude': -118.25703,
 'geometry': {'type': 'Point', 'coordinates': [-118.25703, 34.05513]},
 'country': 'United States',
 'countryCode': 'US',
 'countryFlag': '🇺🇸',
 'county': 'Los Angeles County',
 'distance': 2809386,
 'city': 'Los Angeles',
 'stateCode': 'CA',
 'state': 'California',
 'layer': 'locality',
 'formattedAddress': 'Los Angeles, CA USA',
 'addressLabel': 'Los Angeles'}
 '''

"\n{'latitude': 34.05513,\n 'longitude': -118.25703,\n 'geometry': {'type': 'Point', 'coordinates': [-118.25703, 34.05513]},\n 'country': 'United States',\n 'countryCode': 'US',\n 'countryFlag': '🇺🇸',\n 'county': 'Los Angeles County',\n 'distance': 2809386,\n 'city': 'Los Angeles',\n 'stateCode': 'CA',\n 'state': 'California',\n 'layer': 'locality',\n 'formattedAddress': 'Los Angeles, CA USA',\n 'addressLabel': 'Los Angeles'}\n "

In [129]:
headers = {'Authorization': 'prj_live_pk_4f017205dfaad39af3182404866753da475d1e04'}

In [131]:
coords_lat = []
coords_lon = []
for i, unit in enumerate(units):
    try:
        query = unit.replace('CAL FIRE', 'calfire').replace('NP', 'National Park').replace('BLM', 'Bureau of Land Management').replace(' - ', '-').replace(' FD', ' Fire Department').replace(' - ', '-')
        query += ' California, USA'
        url = f"https://api.radar.io/v1/search/autocomplete?query={query}"
        print(f'{i}.Requesting: {query}')
        response = requests.get(url, headers=headers)
        assert response.status_code == 200, f'error: {response.status_code}'
        assert len(response.json()['addresses'])
    except:
        try:
            query = unit.replace('CAL FIRE', 'calfire').replace('NP', 'National Park').replace('BLM', 'Bureau of Land Management').replace(' - ', '-').replace('calfire','')
            query += ' California, USA'
            url = f"https://api.radar.io/v1/search/autocomplete?query={query}"
            print(f'{i}.Requesting: {query}')
            response = requests.get(url, headers=headers)
            assert response.status_code == 200, f'error: {response.status_code}'
            assert len(response.json()['addresses'])
        except:
            print('got NaN')
            coords_lat.append(0)
            coords_lon.append(0)
            continue
    lat = response.json()['addresses'][0]['latitude']
    lon = response.json()['addresses'][0]['longitude']
    coords_lat.append(lat)
    coords_lon.append(lon)
    print(f'Got: \t{unit}: {lat}, {lon}')

0.Requesting: Los Angeles County California, USA
Got: 	Los Angeles County: 34.05513, -118.25703
1.Requesting: Ventura County California, USA
Got: 	Ventura County: 34.27834, -119.29317
2.Requesting: Riverside calfire California, USA
Got: 	Riverside CAL FIRE: 33.496892, -117.1504
3.Requesting: Shasta-Trinity calfire California, USA
Got: 	Shasta - Trinity CAL FIRE: 40.546793, -122.3855
4.Requesting: San Bernardino National Forest California, USA
Got: 	San Bernardino National Forest: 34.090473, -117.260245
5.Requesting: Plumas National Forest California, USA
Got: 	Plumas National Forest: 39.937816, -120.94114
6.Requesting: Fresno-Kings calfire California, USA
Got: 	Fresno-Kings CAL FIRE: 36.75156, -119.800612
7.Requesting: Humboldt-Del Norte calfire California, USA
Got: 	Humboldt - Del Norte CAL FIRE: 40.641635, -123.926104
8.Requesting: Tahoe National Forest California, USA
Got: 	Tahoe National Forest: 39.301182, -120.678146
9.Requesting: Stanislaus National Forest California, USA
Got: 	S

In [132]:
# coords_lon

In [133]:
units_df = pd.DataFrame(data={'unit':units,'lat':coords_lat,'lon':coords_lon})

In [134]:
units_df

,unit,lat,lon
0,Los Angeles County,34.055130,-118.257030
1,Ventura County,34.278340,-119.293170
2,Riverside CAL FIRE,33.496892,-117.150400
3,Shasta - Trinity CAL FIRE,40.546793,-122.385500
4,San Bernardino National Forest,34.090473,-117.260245
...,...,...,...
75,Kern National Wildlife Refuge,35.739565,-119.620699
76,Sacramento National Wildlife Refuge,39.407675,-122.164103
77,Bitter Creek National Wildlife Refuge,34.959811,-119.314680
78,Los Angeles City FD,34.258426,-118.301548


In [137]:
units_df.to_csv('units_geocoordinates2.csv', index=False)

In [136]:
units_df[units_df.lat==0]

,unit,lat,lon
56,Carson City District or - BLM,0.0,0.0
64,Humbolt-Toiyabe National Forest,0.0,0.0
66,Mojave - NPS,0.0,0.0
70,CRA,0.0,0.0
73,NaN,0.0,0.0


In [135]:
df[df.UNIT_ID.isna()]

,YEAR,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE
11344,2012,USDA Forest Service,NaN,BARRY POINT,120680,2012-08-06 00:00:00,2012-09-17,1 - Lightning,93071.0000,92944.5300,7 - Mixed Collection Methods,Suppression (Wildfire)
11456,2014,Other,NaN,OREGON GULCH,5589,2014-07-30 00:00:00,2014-08-13,1 - Lightning,35105.2137,35111.2500,7 - Mixed Collection Methods,Suppression (Wildfire)
11530,2015,Local Response Area,NaN,LOCUST,17997,2015-07-27 00:00:00,2015-07-27,7 - Arson,NaN,644.1685,6 - Hand Drawn,Suppression (Wildfire)


Notes:
    CRA unit - unknown  
    BARRY POINT - oregon  
    OREGON GULCH - oregon  
    LOCUST - North Sacramento County  
    56	Carson City District or - BLM	0.0	0.0  
64	Humbolt-Toiyabe National Forest	0.0	0.0  
66	Mojave - NPS	0.0	0.0  
70	CRA	0.0	0.0  
73	NaN	0.0	0.0  
    
    
    

In [111]:
df = df.dropna(subset='UNIT_ID')

In [142]:
df = df[~df.UNIT_ID.isin(['CRA', 'Carson City District or - BLM', 'Humbolt-Toiyabe National Forest', 'Mojave - NPS'])]

In [143]:
df

,YEAR,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE
0,1878,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,59468.8800,NaN,Suppression (Wildfire)
1,1895,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,628.1136,NaN,Suppression (Wildfire)
2,1896,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,7531.7170,NaN,Suppression (Wildfire)
3,1898,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,1660.1030,NaN,Suppression (Wildfire)
4,1898,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,428.4387,NaN,Suppression (Wildfire)
...,...,...,...,...,...,...,...,...,...,...,...,...
11910,2017,National Park Service,Yosemite National Park,SOUTH FORK,108,2017-08-13 00:00:00,2017-11-03,1 - Lightning,7000.0,7560.0770,8 - Unknown,Suppression (Wildfire)
11911,2017,California Department of Forestry and Fire Pro...,Sonoma - Lake - Napa CAL FIRE,REDWOOD VALLEY,12172,2017-10-08 00:00:00,2017-10-25,14 - Unknown / Unidentified,NaN,36522.9500,8 - Unknown,Suppression (Wildfire)
11912,2017,USDA Forest Service,Rouge River- Siskiyou,ABNEY,568,2017-08-10 00:00:00,2018-01-10,1 - Lightning,NaN,32893.2600,8 - Unknown,Suppression (Wildfire)
11913,2017,USDA Forest Service,Rouge River- Siskiyou,KNOX,612,2017-08-11 00:00:00,2017-11-28,1 - Lightning,NaN,492.5997,8 - Unknown,Suppression (Wildfire)


## Merge main DF and GeoCoordinates

In [145]:
df = df.merge(units_df, left_on='UNIT_ID',right_on='unit')

In [146]:
df.columns = [x.lower() for x in df.columns]

In [147]:
df

,year,agency,unit_id,fire_name,inc_num,alarm_date,cont_date,cause,report_ac,gis_acres,c_method,objective,unit,lat,lon
0,1878,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,59468.8800,NaN,Suppression (Wildfire),Los Angeles County,34.055130,-118.257030
1,1895,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,628.1136,NaN,Suppression (Wildfire),Los Angeles County,34.055130,-118.257030
2,1896,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,7531.7170,NaN,Suppression (Wildfire),Los Angeles County,34.055130,-118.257030
3,1898,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,1660.1030,NaN,Suppression (Wildfire),Los Angeles County,34.055130,-118.257030
4,1898,Contract County,Los Angeles County,,0,NaN,NaN,14 - Unknown / Unidentified,NaN,428.4387,NaN,Suppression (Wildfire),Los Angeles County,34.055130,-118.257030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11886,2014,USDI Fish and Wildlife Service,Bitter Creek National Wildlife Refuge,SCHOOL,NaN,2014-04-30 00:00:00,2014-05-02,9 - Miscellaneous,224.0,223.7797,1 - GPS Ground,Suppression (Wildfire),Bitter Creek National Wildlife Refuge,34.959811,-119.314680
11887,2017,Local Response Area,Los Angeles City FD,SKIRBALL,30179,2017-12-06 00:00:00,2017-12-07,4 - Campfire,422.0,371.3076,8 - Unknown,Suppression (Wildfire),Los Angeles City FD,34.258426,-118.301548
11888,2017,Contract County,Los Angeles City FD,LA TUNA,965,2017-09-01 00:00:00,2017-09-08,14 - Unknown / Unidentified,7194.0,7051.3510,8 - Unknown,Suppression (Wildfire),Los Angeles City FD,34.258426,-118.301548
11889,2017,USDA Forest Service,Rouge River- Siskiyou,ABNEY,568,2017-08-10 00:00:00,2018-01-10,1 - Lightning,NaN,32893.2600,8 - Unknown,Suppression (Wildfire),Rouge River- Siskiyou,42.273370,-83.134370


In [149]:
df = df[df.lat!=0]

In [150]:
df.to_csv('wildfires_geo2.csv', index=False)